In [2]:
import pythreejs
from ipywidgets import Text, Output, VBox, embed
from IPython.display import display
import numpy as np
import matplotlib
import matplotlib.pylab as plt
from tacs import TACS

# Load the data
loader = TACS.FH5Loader()
loader.loadData('output64.f5')

# Get the colormap from matplotlib
cmap = plt.cm.get_cmap('coolwarm')

# Determine which elements to mask out based on the design variable value
upper = 0.5
val_mask = loader.computeValueMask(TACS.HEXA_ELEMENT, 'dv1', upper=upper)

# Get the iso-surface values
isoval_objs = []
x = np.linspace(0.1, 49.9, 50)
for isoval in x:
    tris = loader.getIsoSurfaces(TACS.HEXA_ELEMENT, isoval, 'X', mask=val_mask)

    # Create the face buffer 
    isoval_pos = pythreejs.BufferAttribute(tris.reshape(-1,3))
    isoval_buff = pythreejs.BufferGeometry(attributes={'position': isoval_pos})

    # Create the threejs objects
    hcolor = matplotlib.colors.to_hex(cmap(isoval/(np.max(x) - np.min(x))))
    isoval_mat = pythreejs.MeshLambertMaterial(side='DoubleSide',
                                               color=hcolor,
                                               transparent=True,
                                               opacity=0.75)
    isoval_objs.append(pythreejs.Mesh(isoval_buff, isoval_mat))
    

# Set the width and height of the view area
view_width = 950
view_height = 400

# Set the target for the camera
target = [25.0, 5.0, 5.0]
camera = pythreejs.PerspectiveCamera(position=[25, 25, 25],
                                     aspect=view_width/view_height)

# Set up both an ambient light
ambient_light = pythreejs.AmbientLight()
key_light = pythreejs.DirectionalLight(position=[-25, -25, -25], intensity=0.7)

# Create the scene and include the objects and light
objs = [ambient_light, key_light]
objs.extend(isoval_objs)
scene = pythreejs.Scene(children=objs, background='black')

# Set an orbit controller 
controller = pythreejs.OrbitControls(controlling=camera, target=target,
                                     minPolarAngle=np.pi/4, maxPolarAngle=3*np.pi/4)

# Render the result
renderer = pythreejs.Renderer(camera=camera, antialias=True,
                              scene=scene, controls=[controller],
                              width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=2.375, position=(25.0, 25.0, 25.0), quaternion=(0.0, 0.0, 0.0, 1.0), …